In [ ]:
##### Copyright 2020 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Demo for NitroML on Cloud using KubeFlow 

## Step 1: Get `kfp` and `skaffold`. 

In [ ]:
import sys

# install kfp (https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.html)
!{sys.executable} -m pip install --user --upgrade -q kfp==0.4.0

# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold && mv skaffold /home/jupyter/.local/bin/
    
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

## Step 2: Check and install  tfx (if necessary)
#### If TFX is not installed, uncomment the pip install command below. We have tested this example with `tfx==0.22.0`

In [ ]:
# !{sys.executable} -m pip install --user --upgrade -q tfx==0.22.0
!python3 -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

## Step 3: Get the GCP project ID and create Docker image name

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GCP_PROJECT_ID=shell_output[0]
print("GCP project ID:" + GCP_PROJECT_ID)

In [ ]:
# Docker image name for the pipeline image 
IMAGE_NAME = 'nitroml_benchmark4'
CUSTOM_TFX_IMAGE='gcr.io/' + GCP_PROJECT_ID + '/' + IMAGE_NAME

## Step 4: Set KFP Cluster End point

In [ ]:
# This refers to the KFP cluster endpoint
# To find your endpoint, go to: Google_Project_Console -> AI_PLATFORMS -> PIPELINES. 
# Then for the cluster you want to run your pipeline on, click on the "Open Pipeline Dashboard". Copy the url "*.googleusercontent.com". This is your ENDPOINT var.
ENDPOINT='ee1a2cabbbc2f13-dot-us-east1.pipelines.googleusercontent.com' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

In [ ]:
import sys, os
PROJECT_DIR=os.path.join(sys.path[0], '..')
%cd {PROJECT_DIR}

In [ ]:
from examples import config
PIPELINE_NAME=config.PIPELINE_NAME

## Step 5: Create the tfx pipeline

In [ ]:
!tfx pipeline create  \
--pipeline-path=examples/titanic_benchmark.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

## Step 6: Run the created tfx pipeline

In [ ]:
!tfx run create --pipeline-name={config.PIPELINE_NAME} --endpoint={ENDPOINT}

## Step 7 (Optional): If the pipeline src is updated, we will have to update the pipeline at endpoint. The following block updates the pipeline and runs it.

In [ ]:
# If we update the pipeline
!tfx pipeline update \
--pipeline-path=examples/titanic_benchmark.py \
--endpoint={ENDPOINT}

!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}